<a href="https://colab.research.google.com/github/superbunny38/SKKUAI/blob/main/model/WideResNetwithSAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataset import random_split
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import KFold
from torch.utils.data import Subset
from torchvision import transforms
from torch.optim import lr_scheduler
import os
import re
import sklearn.preprocessing as preprocessing
from sklearn.preprocessing import normalize
import PIL.Image as Image

In [ ]:
import os
from dataclasses import dataclass
import pandas as pd
import kornia
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm.auto import tqdm
import seaborn as sns

# Get Cow Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Get data from gdrive

In [ ]:
import shutil
!mkdir train_data
#file1 = "/content/drive/MyDrive/ai_competition/train.zip"
file1 = "/content/drive/MyDrive/2022/ai_competition/train.zip"
#file1 = "/content/drive/MyDrive/train.zip"
shutil.unpack_archive(file1, "/content/train_data")

mkdir: cannot create directory ‘train_data’: File exists


In [ ]:
!mkdir train_data2
file2 = "/content/drive/MyDrive/2022/ai_competition/cutmix_train.zip"
#file2 = "/content/drive/MyDrive/ai_competition/cutmix_train.zip"
shutil.unpack_archive(file2, "/content/train_data2")

In [ ]:
!mkdir train_data3
#file3 = "/content/drive/MyDrive/ai_competition/WAtrainset1.zip"
file3 = "/content/drive/MyDrive/2022/ai_competition/WAtrainset1.zip"
shutil.unpack_archive(file3, "/content/train_data3")

In [ ]:
import pandas as pd
train_img_root = "/content/train_data/images/"
train_img_names = os.listdir(train_img_root)
len(train_img_names)

10000

In [ ]:
labels = pd.read_csv('/content/train_data/grade_labels.csv')

In [ ]:
len(labels)

10000

In [ ]:
df=pd.read_csv('/content/stratified_labels.csv')
import sklearn.preprocessing as preprocessing
labelencoder=preprocessing.LabelEncoder()
df['Grade']=labelencoder.fit(['1++','1+','1','2','3']).transform(df['grade'])

train_len=int(len(df))
class Dataset(Dataset):

    def __init__(self, data, path, transform=None):
        self.data=data # data = csv file
        self.path=path # data directory
        self.transform=transform

    def __getitem__(self, idx):
        file_name = self.data['imname'][idx]
        img = Image.open(self.path+file_name)
        label = self.data['Grade'][idx]
        label = torch.tensor(label, dtype=torch.int64)

        if self.transform:
            data = self.transform(img)

        return data, label
    
    def __len__(self):
        return len(self.data)

whole_dataset = Dataset(df,'/content/train_data/images/',transform=train_transform)

In [ ]:
wholeloader = torch.utils.data.DataLoader(whole_dataset, batch_size = cfg.train.batch_size, shuffle = False, num_workers = 0)

In [ ]:
len(whole_dataset)

7205

In [ ]:
train_size = 6305
test_size = len(whole_dataset) - train_size

In [ ]:
print(train_size, test_size)

6305 900


In [ ]:
# Created using indices from 0 to train_size.
train_dataset = torch.utils.data.Subset(whole_dataset, range(train_size))
# Created using indices from train_size to train_size + test_size.
test_dataset = torch.utils.data.Subset(whole_dataset, range(train_size, train_size + test_size))

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=cfg.train.batch_size,
                        shuffle=False, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=cfg.train.batch_size,
                        shuffle=False, num_workers=0)

In [ ]:
# To perform augmentation on GPU, use kornia library.
transform1 = kornia.augmentation.AugmentationSequential(
    #kornia.augmentation.RandomResizedCrop((28, 28)),
    kornia.augmentation.RandomHorizontalFlip(p=0.5),# FILL HERE
    kornia.augmentation.ColorJitter(0.4, 0.4, 0.4, 0, p=0.3),# FILL HERE
    #kornia.augmentation.RandomGrayscale(p=0.2),# FILL HERE
)
transform2 = kornia.augmentation.AugmentationSequential(
    #kornia.augmentation.RandomResizedCrop((28, 28)),
    kornia.augmentation.RandomVerticalFlip(p=0.5),# FILL HERE
    #kornia.augmentation.ColorJitter(0.4, 0.4, 0.4, 0, p=0.3),# FILL HERE
    kornia.augmentation.RandomGrayscale(p=0.2),# FILL HERE
)
transform3 = kornia.augmentation.AugmentationSequential(
    #kornia.augmentation.RandomResizedCrop((28, 28)),
    kornia.augmentation.RandomRotation(90.),# FILL HERE
    #kornia.augmentation.ColorJitter(0.4, 0.4, 0.4, 0, p=0.3),# FILL HERE
    #kornia.augmentation.RandomGrayscale(p=0.2),# FILL HERE
)

## Import SAM

In [ ]:
!git clone https://github.com/davda54/sam.git sam

Cloning into 'sam'...
remote: Enumerating objects: 183, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 183 (delta 63), reused 57 (delta 55), pack-reused 104
Receiving objects: 100% (183/183), 651.88 KiB | 5.98 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [ ]:
pwd

'/content'

In [ ]:
!cp -r '/content/sam/example/utility'  '/content'

In [ ]:
!cp '/content/sam/sam.py'  '/content'

In [ ]:
!cp -r "/content/utility/bypass_bn.py" "/content"

In [ ]:
!cp -r "/content/utility/step_lr.py" "/content"

In [ ]:
!cp -r "/content/utility/initialize.py" "/content"

In [ ]:
!cp -r "/content/utility/log.py" "/content"

In [ ]:
!cp -r "/content/sam/example/model/smooth_cross_entropy.py" "/content"

In [ ]:
!cp -r "/content/sam/sam.py" "/content"

In [ ]:
import torch


class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, adaptive=False, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, adaptive=adaptive, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups
        self.defaults.update(self.base_optimizer.defaults)

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                self.state[p]["old_p"] = p.data.clone()
                e_w = (torch.pow(p, 2) if group["adaptive"] else 1.0) * p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.data = self.state[p]["old_p"]  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        ((torch.abs(p) if group["adaptive"] else 1.0) * p.grad).norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm

    def load_state_dict(self, state_dict):
        super().load_state_dict(state_dict)
        self.base_optimizer.param_groups = self.param_groups


In [ ]:
from log import Log
from initialize import initialize
from step_lr import StepLR
from bypass_bn import enable_running_stats, disable_running_stats
from smooth_cross_entropy import smooth_crossentropy

# Get WideResNet from SAM

In [ ]:
!cp -r "/content/sam/example/model/wide_res_net.py" "/content"

In [ ]:
from wide_res_net import WideResNet

In [ ]:
model = WideResNet(16, 8,0.0, in_channels=3, labels=5).cuda()

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'wide_resnet101_2', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Downloading: "https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet101_2-32ee1156.pth


  0%|          | 0.00/243M [00:00<?, ?B/s]

In [ ]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

In [ ]:
model.fc = nn.Linear(in_features= 2048, out_features = 5)

## Train Function

In [ ]:
def fit(epoch,model,data_loader,phase='train', mode = "linear", backbone = None):
    optimizer = torch.optim.Adam(model.parameters(), lr = cfg.eval.lr, weight_decay = cfg.eval.weight_decay)
    if phase == 'train':
        model.train()
    if phase == 'test':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , (data,target) in enumerate(data_loader):
      
        data,target = data.cuda(),target.cuda()
        
        if phase == 'train':
          optimizer.zero_grad()
        
        output = model(data)
        
        loss = F.cross_entropy(output,target)
        running_loss +=  F.cross_entropy(output,target,size_average=False).data#[0]
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'train':
          loss.backward()
          optimizer.step()
    
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * running_correct/len(data_loader.dataset)
    return model, loss, accuracy

In [ ]:
import copy
def train(cfg, train_loader, test_loader,resnet):
    PATH = "/content/drive/MyDrive/2022/ai_competition/wideresnet/wideresnet.pt"#save simclr model
    torch.autograd.set_detect_anomaly(True)
    pretrain_loss = []#save loss for pretext task
    linear_accuracy = []#test accuracy for linear evaluation
    finetuned_accuracy = []#test accuracy for finetuning
    
    save_test_loss = []
    save_test_acc = []

    resnet.cuda()
    base_optimizer = torch.optim.SGD
    opt = SAM(resnet.parameters(), base_optimizer, rho=2.0, adaptive=True, lr=0.01, momentum=0.9, weight_decay=0.0005)
    #opt = SAM(resnet.parameters(), base_optimizer, lr=0.1, momentum=0.9)
    
    for epoch in range(cfg.train.epoch):
        resnet.train()
        enable_running_stats(resnet)
        train_loss = 0
        with tqdm(train_loader) as pbar:
            pbar.set_description(f"Epoch {epoch + 1}")
            for b_idx, x in enumerate(pbar):
                
                batch = x[0].cuda()
                labels = x[1].cuda()
                
                #cutmix
                lam = np.random.beta(1.0, 1.0)
                rand_index = torch.randperm(batch.size()[0])
                shuffled_labels = labels[rand_index]
                bbx1, bby1, bbx2, bby2 = cut(batch.shape[2], batch.shape[3], lam) # define a box to cut and mix
                batch[:, :, bbx1:bbx2, bby1:bby2] = batch[rand_index, :, bbx1:bbx2, bby1:bby2].clone() #cut and mix
                lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (batch.shape[-1] * batch.shape[-2]))
                #cutmix
                enable_running_stats(resnet)
                output = resnet(batch)
                loss =smooth_crossentropy(output,labels)*lam + smooth_crossentropy(output, shuffled_labels)*(1.0-lam)
                loss.clone().mean().backward(retain_graph=True)
                opt.first_step(zero_grad = True)
                #print("hi")

                disable_running_stats(resnet)
                output,labels,shuffled_labels = output.clone(),labels.clone(),shuffled_labels.clone()
                loss = smooth_crossentropy(output,labels)*lam 
                loss = loss.clone()+ smooth_crossentropy(output.clone(), shuffled_labels.clone())*(1.0-lam)
                loss.clone().mean().backward(retain_graph=True)# make sure to do a full forward pass
                
                opt.second_step(zero_grad=True)
                #opt.zero_grad()

                with torch.no_grad():
                  train_loss += loss.detach().cpu().numpy()
                  pbar.set_postfix(loss=train_loss / (b_idx + 1))
        pretrain_loss.append(train_loss/(b_idx+1))
        if (epoch + 1) % 2 == 0:
          resnet, test_loss, test_acc = fit(epoch, resnet, test_loader, phase = "test", mode = 'finetune')
          print(f"test loss: {test_loss}, test_acc: {test_acc}")
          save_test_loss.append(test_loss)
          save_test_acc.append(test_acc)
          torch.save({"resnet":resnet.state_dict()},PATH)#save optimizer
          
        
    torch.save({"resnet":resnet.state_dict()},PATH)#save optimizer
    return resnet, save_test_loss, save_test_acc#return pretrainloss, and two accuracy for downstream task

In [ ]:
resnet, save_test_loss, save_test_acc = train(cfg,train_loader,test_loader,model)

  0%|          | 0/2400 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/autograd/__init__.py:175: UserWarning: Error detected in AddmmBackward0. Traceback of forward call that caused the error:
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.7/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.7/asyncio/base_events.py", line 541, in run_forever
    self._run_once()
  File "/usr/lib/python3.7/asyncio/base_events.py", li

RuntimeError: ignored

In [ ]:
import torch
print(torch.cuda.memory_allocated()/1024**2)
import gc
gc.collect()
a = torch.cuda.empty_cache() 
print(torch.cuda.memory_allocated()/1024**2)

14964.4775390625
14964.4775390625


# Configuration settings

In [ ]:
@dataclass
class Args:
    adaptive: bool = True# FILL HERE: optimal value stated in the paper
    batch_size: int = 10# FILL HERE : I changed from 30 -> 60 -> 100
    depth: int = 16# FILL HERE: The bigger, the better but I used frequently-used number 
    dropout: float = 0.0# FILL HERE: I put small number so that I can check how the accuracy for downstream change in detail
    epochs: int = 200
    label_smoothing: float = 0.1
    learning_rate: float = 0.1
    momentum: float = 0.9    
    threads: int = 2
    rho: int = 2
    weight_decay: float = 0.0005
    width_factor: int = 8

    opt: str = "adam"
    lr: float = 1e-3
    weight_decay: float = 1e-6

args = Args()

In [ ]:
!git clone https://github.com/superbunny38/sam_cow.git sam

Cloning into 'sam'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 196 (delta 70), reused 56 (delta 54), pack-reused 104
Receiving objects: 100% (196/196), 655.29 KiB | 4.37 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [ ]:
!cp -r "/content/sam/example/data/cifar.py" "/content"

In [ ]:
!cp -r "/content/sam/example/train.py" "/content"

In [ ]:
!python train.py

Traceback (most recent call last):
  File "train.py", line 4, in <module>
    from model.wide_res_net import WideResNet
ModuleNotFoundError: No module named 'model'
^C


In [ ]:
!mkdir new_wd

In [ ]:
%cd new_wd

/content/new_wd


# Clone SAM + WideResNet github repo that is modified for our dataset

In [ ]:
!git clone https://github.com/superbunny38/sam_cow.git sam_cow

Cloning into 'sam_cow'...
remote: Enumerating objects: 192, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 192 (delta 68), reused 56 (delta 54), pack-reused 104
Receiving objects: 100% (192/192), 654.40 KiB | 5.59 MiB/s, done.
Resolving deltas: 100% (90/90), done.


In [ ]:
!ls

sam_cow


In [ ]:
%cd sam_cow

/content/new_wd/sam_cow


In [ ]:
!ls

example  img  LICENSE  README.md  sam.py


In [ ]:
%cd example/

/content/new_wd/sam_cow/example


In [ ]:
!ls

data  model  README.md	train.py  utility


In [ ]:
%cd sam

/content/sam


In [ ]:
%cd example

[Errno 2] No such file or directory: 'example'
/content/sam/example


In [ ]:
pwd

'/content/sam/example'

In [ ]:
sum([False,False,True,True,True])

3

## Train Log1

In [ ]:
!python3 train.py

┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨
┃           0  ┃      1.1549  │     19.00 %  ┃   1.000e-03  │   00:36 min  ┠┈███████████████████████████┈┨train correct: 1756 train_total_size 24000
┃           0  ┃      1.1460  │     21.95 %  ┃   1.000e-03  │   00:36 min  ┃test correct: 446 total: 6000
      1.1454  │     22.30 %  ┃
┃           1  ┃      1.1484  │     22.00 %  ┃   1.000e-03  │   00:35 min  ┠┈███████████████████████████┈┨train correct: 1756 train_total_size 24000
┃           1  ┃      1.1462  │     21.95 %  ┃   1.000e-03  │   00:35 min  ┃test correct: 431 total: 6000
      1.1432  │     21.55 %  ┃


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/2022/ai_competition/train_data/grade_labels.csv')
df.head()

,imname,grade
0,cow_1++_4567.jpg,1++
1,cow_2_1390.jpg,2
2,cow_1++_2581.jpg,1++
3,cow_2_1689.jpg,2
4,cow_3_3287.jpg,3


In [ ]:
import sklearn.preprocessing as preprocessing
labelencoder=preprocessing.LabelEncoder()
df['Grade']=labelencoder.fit(['1++','1+','1','2','3']).transform(df['grade'])

In [ ]:
df.head()

,imname,grade,Grade
0,cow_1++_4567.jpg,1++,2
1,cow_2_1390.jpg,2,3
2,cow_1++_2581.jpg,1++,2
3,cow_2_1689.jpg,2,3
4,cow_3_3287.jpg,3,4


In [ ]:
df.to_csv('/content/drive/MyDrive/2022/ai_competition/train_data/grade_labels.csv',index=False)

In [ ]:
!git clone https://github.com/davda54/sam.git real_sam

Cloning into 'real_sam'...
remote: Enumerating objects: 183, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 183 (delta 63), reused 56 (delta 54), pack-reused 104
Receiving objects: 100% (183/183), 651.88 KiB | 3.97 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [ ]:
%cd real_sam

/content/real_sam


In [ ]:
!cp -r /content/train_data "/content/drive/MyDrive/2022/ai_competition"

In [ ]:
pwd

'/content/real_sam'

In [ ]:
%cd example

/content/real_sam/example


## Train Log2

In [ ]:
!python3 train.py

┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨
┃           0  ┃      1.1395  │     21.48 %  ┃   1.000e-03  │   00:20 min  ┃      1.1402  │     21.80 %  ┃
┃           1  ┃      1.1383  │     21.61 %  ┃   1.000e-03  │   00:19 min  ┃      1.1394  │     21.55 %  ┃
┃           2  ┃      1.1383  │     21.06 %  ┃   1.000e-03  │   00:19 min  ┃      1.1401  │     21.55 %  ┃
┃           3  ┃      1.1384  │     21.43 %  ┃   1.000e-03  │   00:19 min  ┃      1.1399  │     20.50 %  ┃
┃           4  ┃      1.1383  │     21.36 %  ┃   1.000e-03  │   00:19 min  ┃      1.1398  │     20.35 %  ┃
┃           5  ┃      1.1381  │     2